AI-Powered Study Assistant using Retrieval-Augmented Generation (RAG)

Course:Computer Networks  

This project implements an AI-powered study assistant designed to help students
answer questions from Computer Networks course materials using Retrieval-Augmented Generation (RAG).

The system processes academic documents such as lecture notes and textbooks,
retrieves relevant content using vector similarity search, and generates
answers using an open-source language model.

Technology Stack

This project is implemented entirely using open-source tools:

Language Model: Mistral (via Ollama, running locally)
Embedding Model: Sentence-Transformers (`all-MiniLM-L6-v2`)
Vector Database: ChromaDB (embedded, local)
Document Processing:PDF-based academic materials
Environment: Jupyter Notebook (Python)

Open-source models were chosen to avoid API costs and to better understand
the practical constraints of local deployment.

Part 1: Data Collection and Understanding

1.1 Dataset Overview

For this project, I collected academic materials from my Computer Networks course.
The dataset consists of lecture notes and reference material in PDF format, covering multiple layers of the network stack.

Types of documents:
- Lecture slide PDFs provided during coursework
- Reference-style notes explaining networking concepts
- Text-heavy PDFs with occasional diagrams and tables

The documents primarily cover the following topics:
- OSI and TCP/IP reference models
- Physical and Data Link layers
- Network layer concepts such as IP and routing
- Transport layer protocols including TCP and UDP

1.2 Document Structure and Formatting

Most documents follow a semi-structured format with headings, bullet points,
and short explanatory paragraphs. However, the structure is not consistent
across all PDFs.

Some documents are slide-based with minimal text per page, while others are
dense text documents resembling textbook chapters. Diagrams are often embedded
as images, and tables are sometimes split across pages.

1.3 Observed Challenges in the Dataset

After inspecting the raw PDFs, I observed several challenges that affect
automatic text processing:

1. Inconsistent formatting: Different PDFs use different heading styles, making it difficult to rely on document structure alone.
2. Broken text flow: In some cases, sentences are split across lines or pages
   during text extraction.
3. Tables and diagrams: Tables are converted into plain text with lost alignment,
   and diagrams do not contain meaningful extractable text.
4. Technical terminology: Networking concepts include abbreviations and protocol
   names (e.g., TCP, UDP, ARP) that require accurate retrieval to avoid confusion.

These challenges reflect real-world academic data and motivate the need for
careful chunking, retrieval, and prompt design in later stages of the project.

In [1]:
import pdfplumber
from pathlib import Path

data_path = Path("../data/raw")

pdf_files = list(data_path.glob("*.pdf"))
print(f"Found {len(pdf_files)} PDF files")

sample_pdf = pdf_files[0]
print(f"Inspecting file: {sample_pdf.name}")

with pdfplumber.open(sample_pdf) as pdf:
    first_page = pdf.pages[0]
    text = first_page.extract_text()

print("----- Extracted Text (First Page) -----")
print(text[:1500]) 

ModuleNotFoundError: No module named 'pdfplumber'